# Compartimental Model Simulator

**Created by:** Samuel Ropert  
**Creation date:** 04/08/2020  
**Institution:** Computational Biology Lab - Fundación Ciencia y Vida, Chile  

## SEIR
This jupyter notebook shows how to build a single SEIR simulation object with different quarantine scenarios. In this example we show 4 quarantine scenarios, 2 with total quarantines, and 2 with dynamic quarantines.


## Import Libraries

In [17]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIR/')
sys.path.insert(1, '../src/utils/')

from datetime import datetime
import numpy as np
from numpy import linalg as LA
import pandas as pd
from time import time

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')


from class_SEIR import SEIR
from Quarantine import Quarantine

Linux


## Epidemiological Parameters
* **beta:** Infection rate
* **mu:** Initial exposed obtained from the initial infected mu=E0/I0
* **Scale Factor:** Proportion of real infected compared to reported ones (1: all the infecteds are reported)
* **Sero Prevalence Factor:** Adjust the proportion of the population that enters the virus dynamics
* **Exposed Infection:** rate compared to the infected (0 the don't infect, 1 the infect in the same rate as the infected )

## Simulation Parameters

In [21]:
beta = 0.2 # Contagion rate
mu = 1.5 # E0/I0 initial rate
SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 500
# Population
population = 1000000
# Initial Active Infected 
I0 = 100

I_ac0 = 100
# Kinetic Saturation: 0 for mass action mixing
k=0

### Quarantines

Quarantine object constructor:
 
 
 ```Quarantine(rem_mov,max_mov=0.85,qp=0,iqt=0,fqt=1000,movfunct = 'once')```
 
 * rem_mov: Remanent mobility during Quarantine
 * max_mov: Mobility during non quarantine periods
 * qp: Quarantine period (for dynamic quarantines)
 * iqt: Initial quarantine time
 * fqt: Final quarantine time
 * movfunct: Mobility function 

Mobility function types:
 * once: Total quarantine between iqt and fqt
 * square: Periodic quaratine with qp period

In [22]:
rem_mob = 0.4 # Mobility during quarantine
max_mob = 0.8 # Maximum mobility
# Total quarantine

s1 = Quarantine(0.5)
s2 = Quarantine(0.5,0.8,qp=14,iqt=50,fqt = 500,movfunct='square')
s3 = Quarantine(0.5,0.8,qp=14,iqt=50,fqt = 500,movfunct='sawtooth')

quarantines = [s1.alpha,s2.alpha,s3.alpha]

## Create simulation Object

In [8]:
simulation = []
for i in range(len(quarantines)):
    simulation.append(SEIR(tsim=tsim,alpha=quarantines[i],beta=beta,mu=mu,k=k,I=I0,population=population,expinfection=1,SeroPrevFactor=1))
#simulation = SEIR(tsim=tsim,alpha=s2.alpha,beta=beta,mu=mu,k=k,I0=I0,population=population,expinfection=1,SeroPrevFactor=1)

## Simulate
The different scenarios are simulated in parallel threads 

In [9]:
#simulation = []
for i in range(len(quarantines)):
    simulation[i].integr_sci(0,tsim,0.01)

# Simulation Analysis

## Plots
This libraries have predefined plot functions which plot the main epidemiological variables.
Each function has the following optional arguments:
* days [int] Number of days to display
* showparams [bool] Display simulation parameters 
* ylim [int] Limit the vertical axis
* norm [int/float] Normalize the results


### SEIR Plot with Active infected

In [10]:
plt.plot(simulation[0].t,simulation[0].S,label='S',color = 'blue')
plt.plot(simulation[0].t,simulation[0].E,label='E',color = 'cyan')
plt.plot(simulation[0].t,simulation[0].I,label='I',color = 'red')
plt.plot(simulation[0].t,simulation[0].R,label='R',color = 'green')
plt.title('Epidemiological Plot')
plt.legend(loc=0)
plt.show()

Plot Active Infected

In [11]:
for i in range(len(quarantines)):
    plt.plot(simulation[i].t,simulation[i].I,label='Quarantine: '+str(i))
plt.title('Quarantines')
plt.legend(loc=0)
plt.show()

### Plot Accumulated Infected

In [12]:
for i in range(len(quarantines)):
    plt.plot(simulation[i].t,simulation[i].I_ac,label='Quarantine: '+str(i))

plt.title('Accumulated Infected')
plt.legend(loc=0)
plt.show()

### Plot New Daily Infected

In [13]:
for i in range(len(quarantines)):
    plt.plot(simulation[i].t,simulation[i].I_d,label='Quarantine: '+str(i))

plt.title('New Daily Infected')
plt.legend(loc=0)
plt.show()

### Variables:
The simulation object contains several methods and variables with its results

In [14]:
# Susceptibles
simulation[0].S
# Exposed
simulation[0].E
# Infected
simulation[0].I
# Recovered
simulation[0].R

array([0.00000000e+00, 1.12938511e-05, 2.25877045e-05, 5.33316552e-02,
       1.06691788e-01, 1.60102946e-01, 6.97012561e-01, 1.23898182e+00,
       1.78597253e+00, 2.33794689e+00, 5.09679366e+00, 7.96890454e+00,
       1.09504366e+01, 1.40378087e+01, 1.72276913e+01, 2.29859534e+01,
       2.90280105e+01, 3.53411259e+01, 4.19141810e+01, 4.87375149e+01,
       5.74031531e+01, 6.64172760e+01, 7.57693169e+01, 8.54508180e+01,
       9.55281373e+01, 1.05928086e+02, 1.16646937e+02, 1.23580054e+02,
       1.30638990e+02, 1.37823573e+02, 1.53784787e+02, 1.70335746e+02,
       1.87481214e+02, 2.54810119e+02, 3.30669020e+02, 3.51029766e+02,
       3.71980415e+02, 3.93541788e+02, 4.38588052e+02, 4.86293859e+02,
       5.36799724e+02, 6.56236566e+02, 7.91166746e+02, 9.43650644e+02,
       1.11595419e+03, 1.46768625e+03, 1.90192627e+03, 2.43765729e+03,
       3.09819557e+03, 4.11559003e+03, 5.42536607e+03, 7.11034935e+03,
       9.27452650e+03, 1.20471825e+04, 1.70505054e+04, 2.39586233e+04,
      

## Peak Calculation

In [15]:
# Peak Values per each scenario
simulation[0].peak

53737.39093475814

In [16]:
# Peak day counted from the simulation beginning
simulation[0].peak_t

182.68419391442006